In [34]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import tenseal as ts 
from os.path import exists
import base64

pd.options.display.float_format = "{:,.4f}".format

In [35]:
FILENAME = "mnist.pkl.gz"
DATA_PATH = Path("./")
PATH = DATA_PATH 
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = pickle.load(f, encoding="latin-1")

# For homomorphic encryption
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

# For storing public and private keys
def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

if exists("./secret.txt") & exists("./public.txt"):
    public_context = ts.context_from(read_data("public.txt"))
    secret_context = ts.context_from(read_data("secret.txt"))
else:
    # Store public and private keys
    secret_context = context.serialize(save_secret_key = True)
    write_data("secret.txt", secret_context)
    
    context.make_context_public() #drop the secret_key from the context
    public_context = context.serialize()
    write_data("public.txt", public_context)

In [36]:
def split_and_shuffle_labels(y_data, seed, amount):
    y_data=pd.DataFrame(y_data,columns=["labels"])
    y_data["i"]=np.arange(len(y_data))
    label_dict = dict()
    for i in range(10):
        var_name="label" + str(i)
        label_info=y_data[y_data["labels"]==i]
        np.random.seed(seed)
        label_info=np.random.permutation(label_info)
        label_info=label_info[0:amount]
        label_info=pd.DataFrame(label_info, columns=["labels","i"])
        label_dict.update({var_name: label_info })
    return label_dict

In [37]:
def get_iid_subsamples_indices(label_dict, number_of_samples, amount):
    sample_dict= dict()
    batch_size=int(math.floor(amount/number_of_samples))
    for i in range(number_of_samples):
        sample_name="sample"+str(i)
        dumb=pd.DataFrame()
        for j in range(10):
            label_name=str("label")+str(j)
            a=label_dict[label_name][i*batch_size:(i+1)*batch_size]
            dumb=pd.concat([dumb,a], axis=0)
        dumb.reset_index(drop=True, inplace=True)    
        sample_dict.update({sample_name: dumb}) 
    return sample_dict

In [38]:
def create_iid_subsamples(sample_dict, x_data, y_data, x_name, y_name):
    x_data_dict= dict()
    y_data_dict= dict()
    
    for i in range(len(sample_dict)):  ### len(sample_dict)= number of samples
        xname= x_name+str(i)
        yname= y_name+str(i)
        sample_name="sample"+str(i)
        
        indices=np.sort(np.array(sample_dict[sample_name]["i"]))
        
        x_info= x_data[indices,:]
        x_data_dict.update({xname : x_info})
        
        y_info= y_data[indices]
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict

In [39]:
#Classification Model

In [40]:
class Net2nn(nn.Module):
    def __init__(self):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(784,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,10)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [41]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [42]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [43]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [44]:
#Optimizers are algorithms or methods used to minimize an
# error function(loss function)or to maximize the efficiency of production. 

# This function creates a model, optimizer and loss function for each node.
def create_model_optimizer_criterion_dict(number_of_samples):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_samples):
        model_name="model"+str(i)
        model_info=Net2nn()
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
        print("MYYYY MODELLL DICT", model_dict)
    return model_dict, optimizer_dict, criterion_dict

In [45]:
# This function should replace the main_model parameters to each local_model parameters
def download_main_model_for_each_node(model_dict, number_of_samples):
    with torch.no_grad():
        for i in range(number_of_samples):
            model_dict[name_of_models[i]].fc1.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc1_weight_data.pt')
            model_dict[name_of_models[i]].fc2.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc2_weight_data.pt')
            model_dict[name_of_models[i]].fc3.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc3_weight_data.pt')
            
            model_dict[name_of_models[i]].fc1.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc1_bias_data.pt')
            model_dict[name_of_models[i]].fc2.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc2_bias_data.pt')
            model_dict[name_of_models[i]].fc3.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc3_bias_data.pt')
    return model_dict 

def download_encrypted_average_main_model(model_dict, number_of_samples):
    # Clients has the secret key
    context = ts.context_from(read_data("secret.txt"))
    
    for i in range(200):
        temp = read_data("Main_Model_Parameters/encrypted_model_1_weight_" + str(i))
        encrypted_model_1_weight = ts.lazy_ckks_vector_from(temp)
        encrypted_model_1_weight.link_context(context)
        model_dict["model0"].fc1.weight[i,:].data[:] = torch.as_tensor(encrypted_model_1_weight.decrypt())
        model_dict["model1"].fc1.weight[i,:].data[:] = torch.as_tensor(encrypted_model_1_weight.decrypt())
        model_dict["model2"].fc1.weight[i,:].data[:] = torch.as_tensor(encrypted_model_1_weight.decrypt())

        temp = read_data("Main_Model_Parameters/encrypted_model_2_weight_" + str(i))
        encrypted_model_2_weight = ts.lazy_ckks_vector_from(temp)
        encrypted_model_2_weight.link_context(context)
        model_dict["model0"].fc2.weight[i,:].data[:] = torch.tensor(encrypted_model_2_weight.decrypt())
        model_dict["model1"].fc2.weight[i,:].data[:] = torch.tensor(encrypted_model_2_weight.decrypt())
        model_dict["model2"].fc2.weight[i,:].data[:] = torch.tensor(encrypted_model_2_weight.decrypt())
        
    for i in range(10):
        temp = read_data("Main_Model_Parameters/encrypted_model_3_weight_" + str(i))
        encrypted_model_3_weight = ts.lazy_ckks_vector_from(temp)
        encrypted_model_3_weight.link_context(context)
        model_dict["model0"].fc3.weight[i,:].data[:] = torch.tensor(encrypted_model_3_weight.decrypt())
        model_dict["model1"].fc3.weight[i,:].data[:] = torch.tensor(encrypted_model_3_weight.decrypt())
        model_dict["model2"].fc3.weight[i,:].data[:] = torch.tensor(encrypted_model_3_weight.decrypt())

    temp = read_data("Main_Model_Parameters/encrypted_model_1_bias")
    encrypted_model_1_bias = ts.lazy_ckks_vector_from(temp)
    encrypted_model_1_bias.link_context(context)
    model_dict["model0"].fc1.bias.data[:] = torch.as_tensor(encrypted_model_1_bias.decrypt())
    model_dict["model1"].fc1.bias.data[:] = torch.as_tensor(encrypted_model_1_bias.decrypt())
    model_dict["model2"].fc1.bias.data[:] = torch.as_tensor(encrypted_model_1_bias.decrypt())

    temp = read_data("Main_Model_Parameters/encrypted_model_2_bias")
    encrypted_model_2_bias = ts.lazy_ckks_vector_from(temp)
    encrypted_model_2_bias.link_context(context)
    model_dict["model0"].fc2.bias.data[:] = torch.as_tensor(encrypted_model_2_bias.decrypt())
    model_dict["model1"].fc2.bias.data[:] = torch.as_tensor(encrypted_model_2_bias.decrypt())
    model_dict["model2"].fc2.bias.data[:] = torch.as_tensor(encrypted_model_2_bias.decrypt())

    temp = read_data("Main_Model_Parameters/encrypted_model_3_bias")
    encrypted_model_3_bias = ts.lazy_ckks_vector_from(temp)
    encrypted_model_3_bias.link_context(context)
    model_dict["model0"].fc3.bias.data[:] = torch.as_tensor(encrypted_model_3_bias.decrypt())
    model_dict["model1"].fc3.bias.data[:] = torch.as_tensor(encrypted_model_3_bias.decrypt())
    model_dict["model2"].fc3.bias.data[:] = torch.as_tensor(encrypted_model_3_bias.decrypt())

    return model_dict

In [46]:
# This function trains individual local models in nodes.
def start_train_end_node_process(number_of_samples):
    for i in range (number_of_samples): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [47]:
# This function trains individual local models in nodes.
def start_train_end_node_process_without_print(number_of_samples):
    for i in range (number_of_samples): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)



In [48]:
# This function trains individual local models in nodes.
def start_train_end_node_process_print_some(number_of_samples, print_amount):
    for i in range (number_of_samples): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))



In [49]:
# This function sends the parameters of the each local_model to the SERVER.
# So we should save them on a file

def send_data_from_local_models_to_server(model_dict, number_of_samples):
    with torch.no_grad():
        for i in range(number_of_samples):
            torch.save(model_dict[name_of_models[i]].fc1.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc1_weight_data.pt')
            torch.save(model_dict[name_of_models[i]].fc1.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc1_bias_data.pt')

            torch.save(model_dict[name_of_models[i]].fc2.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc2_weight_data.pt')
            torch.save(model_dict[name_of_models[i]].fc2.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc2_bias_data.pt')

            torch.save(model_dict[name_of_models[i]].fc3.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc3_weight_data.pt')
            torch.save(model_dict[name_of_models[i]].fc3.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc3_bias_data.pt')

In [50]:
x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))
number_of_samples = 3
learning_rate = 0.01
numEpoch = 10
batch_size = 32
momentum = 0.9

train_amount = 4500
valid_amount = 900
test_amount = 900
print_amount = 3

In [51]:
# Data is distributed to nodes


In [52]:
label_dict_train=split_and_shuffle_labels(y_data=y_train, seed=1, amount=train_amount) 
sample_dict_train=get_iid_subsamples_indices(label_dict=label_dict_train, number_of_samples=number_of_samples, amount=train_amount)
x_train_dict, y_train_dict = create_iid_subsamples(sample_dict=sample_dict_train, x_data=x_train, y_data=y_train, x_name="x_train", y_name="y_train")

label_dict_valid = split_and_shuffle_labels(y_data=y_valid, seed=1, amount=train_amount) 
sample_dict_valid = get_iid_subsamples_indices(label_dict=label_dict_valid, number_of_samples=number_of_samples, amount=valid_amount)
x_valid_dict, y_valid_dict = create_iid_subsamples(sample_dict=sample_dict_valid, x_data=x_valid, y_data=y_valid, x_name="x_valid", y_name="y_valid")

label_dict_test = split_and_shuffle_labels(y_data=y_test, seed=1, amount=test_amount) 
sample_dict_test = get_iid_subsamples_indices(label_dict=label_dict_test, number_of_samples=number_of_samples, amount=test_amount)
x_test_dict, y_test_dict = create_iid_subsamples(sample_dict=sample_dict_test, x_data=x_test, y_data=y_test, x_name="x_test", y_name="y_test")

In [53]:
# Models,optimizers and loss functions in nodes are defined


In [54]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_samples)

MYYYY MODELLL DICT {'model0': Net2nn(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)}
MYYYY MODELLL DICT {'model0': Net2nn(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
), 'model1': Net2nn(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)}
MYYYY MODELLL DICT {'model0': Net2nn(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
), 'model1': Net2nn(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out

In [55]:
# Keys of dicts are being made iterable


In [56]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())
name_of_x_valid_sets=list(x_valid_dict.keys())
name_of_y_valid_sets=list(y_valid_dict.keys())
name_of_x_test_sets=list(x_test_dict.keys())
name_of_y_test_sets=list(y_test_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

# print(name_of_x_train_sets)
# print(name_of_y_train_sets)
# print(name_of_x_valid_sets)
# print(name_of_y_valid_sets)
# print(name_of_x_test_sets)
# print(name_of_y_test_sets)
# print("\n ------------")
# print(name_of_models)
# print(name_of_optimizers)
# print(name_of_criterions)

In [57]:
print(model_dict["model1"].fc2.weight[0:1,0:5])
print(model_dict["model0"].fc2.weight[0:1,0:5])

tensor([[-0.0447, -0.0437, -0.0335, -0.0376, -0.0611]],
       grad_fn=<SliceBackward0>)
tensor([[ 0.0518,  0.0474,  0.0405,  0.0644, -0.0257]],
       grad_fn=<SliceBackward0>)


In [58]:
# Parameters of main_model should be downloaded by each node
model_dict = download_main_model_for_each_node(model_dict, number_of_samples)

# model_dict = download_encrypted_average_main_model(model_dict, number_of_samples)

In [59]:
print(model_dict["model1"].fc2.weight[0:1,0:5])
print(model_dict["model0"].fc2.weight[0:1,0:5])

tensor([[-0.0616,  0.0281, -0.0098, -0.0555,  0.0535]],
       grad_fn=<SliceBackward0>)
tensor([[-0.0616,  0.0281, -0.0098, -0.0555,  0.0535]],
       grad_fn=<SliceBackward0>)


In [60]:
# Models in the nodes are trained


In [61]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_samples, print_amount)

Subset 0
epoch:   1 | train accuracy: 0.76927 | test accuracy: 0.89700
epoch:   2 | train accuracy: 0.91207 | test accuracy: 0.92567
epoch:   3 | train accuracy: 0.93720 | test accuracy: 0.93467
epoch:   4 | train accuracy: 0.95233 | test accuracy: 0.94000
epoch:   5 | train accuracy: 0.96433 | test accuracy: 0.94467
epoch:   6 | train accuracy: 0.96973 | test accuracy: 0.95667
epoch:   7 | train accuracy: 0.97660 | test accuracy: 0.95967
epoch:   8 | train accuracy: 0.98080 | test accuracy: 0.95567
epoch:   9 | train accuracy: 0.98607 | test accuracy: 0.95967
epoch:  10 | train accuracy: 0.98980 | test accuracy: 0.96167
Subset 1
epoch:   1 | train accuracy: 0.76267 | test accuracy: 0.91233
epoch:   2 | train accuracy: 0.91840 | test accuracy: 0.93400
epoch:   3 | train accuracy: 0.94027 | test accuracy: 0.94533
epoch:   4 | train accuracy: 0.95560 | test accuracy: 0.95000
epoch:   5 | train accuracy: 0.96693 | test accuracy: 0.95500
epoch:   6 | train accuracy: 0.97140 | test accuracy

In [62]:
from torch import float64

encrypted_model_0_1_weight = [ts.ckks_vector(context, model_dict["model0"].fc1.weight[i,:].data) for i in range(200)]
encrypted_model_0_1_bias   = ts.ckks_vector(context, model_dict["model0"].fc1.bias[:].data)
encrypted_model_0_2_weight = [ts.ckks_vector(context, model_dict["model0"].fc2.weight[i,:].data) for i in range(200)]
encrypted_model_0_2_bias   = ts.ckks_vector(context, model_dict["model0"].fc2.bias[:].data)
encrypted_model_0_3_weight = [ts.ckks_vector(context, model_dict["model0"].fc3.weight[i,:].data) for i in range(10)]
encrypted_model_0_3_bias   = ts.ckks_vector(context, model_dict["model0"].fc3.bias[:].data)

encrypted_model_1_1_weight = [ts.ckks_vector(context, model_dict["model1"].fc1.weight[i,:].data) for i in range(200)]
encrypted_model_1_1_bias   = ts.ckks_vector(context, model_dict["model1"].fc1.bias[:].data)
encrypted_model_1_2_weight = [ts.ckks_vector(context, model_dict["model1"].fc2.weight[i,:].data) for i in range(200)]
encrypted_model_1_2_bias   = ts.ckks_vector(context, model_dict["model1"].fc2.bias[:].data)
encrypted_model_1_3_weight = [ts.ckks_vector(context, model_dict["model1"].fc3.weight[i,:].data) for i in range(10)]
encrypted_model_1_3_bias   = ts.ckks_vector(context, model_dict["model1"].fc3.bias[:].data)

encrypted_model_2_1_weight = [ts.ckks_vector(context, model_dict["model2"].fc1.weight[i,:].data) for i in range(200)]
encrypted_model_2_1_bias   = ts.ckks_vector(context, model_dict["model2"].fc1.bias[:].data)
encrypted_model_2_2_weight = [ts.ckks_vector(context, model_dict["model2"].fc2.weight[i,:].data) for i in range(200)]
encrypted_model_2_2_bias   = ts.ckks_vector(context, model_dict["model2"].fc2.bias[:].data)
encrypted_model_2_3_weight = [ts.ckks_vector(context, model_dict["model2"].fc3.weight[i,:].data) for i in range(10)]
encrypted_model_2_3_bias   = ts.ckks_vector(context, model_dict["model2"].fc3.bias[:].data)

for i in range(200):
    encrypted_model_0_1_weight[i] = encrypted_model_0_1_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_0_1_weight_' + str(i), encrypted_model_0_1_weight[i])
    encrypted_model_1_1_weight[i] = encrypted_model_1_1_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_1_1_weight_' + str(i), encrypted_model_1_1_weight[i])
    encrypted_model_2_1_weight[i] = encrypted_model_2_1_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_2_1_weight_' + str(i), encrypted_model_2_1_weight[i])

    encrypted_model_0_2_weight[i] = encrypted_model_0_2_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_0_2_weight_' + str(i), encrypted_model_0_2_weight[i])
    encrypted_model_1_2_weight[i] = encrypted_model_1_2_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_1_2_weight_' + str(i), encrypted_model_1_2_weight[i])
    encrypted_model_2_2_weight[i] = encrypted_model_2_2_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_2_2_weight_' + str(i), encrypted_model_2_2_weight[i])

for i in range(10):
    encrypted_model_0_3_weight[i] = encrypted_model_0_3_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_0_3_weight_' + str(i), encrypted_model_0_3_weight[i])
    encrypted_model_1_3_weight[i] = encrypted_model_1_3_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_1_3_weight_' + str(i), encrypted_model_1_3_weight[i])
    encrypted_model_2_3_weight[i] = encrypted_model_2_3_weight[i].serialize()
    write_data('Local_Model_Parameters/encrypted_model_2_3_weight_' + str(i), encrypted_model_2_3_weight[i])

encrypted_model_0_1_bias = encrypted_model_0_1_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_0_1_bias', encrypted_model_0_1_bias)
encrypted_model_0_2_bias = encrypted_model_0_2_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_0_2_bias', encrypted_model_0_2_bias)
encrypted_model_0_3_bias = encrypted_model_0_3_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_0_3_bias', encrypted_model_0_3_bias)

encrypted_model_1_1_bias = encrypted_model_1_1_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_1_1_bias', encrypted_model_1_1_bias)
encrypted_model_1_2_bias = encrypted_model_1_2_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_1_2_bias', encrypted_model_1_2_bias)
encrypted_model_1_3_bias = encrypted_model_1_3_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_1_3_bias', encrypted_model_1_3_bias)

encrypted_model_2_1_bias = encrypted_model_2_1_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_2_1_bias', encrypted_model_2_1_bias)
encrypted_model_2_2_bias = encrypted_model_2_2_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_2_2_bias', encrypted_model_2_2_bias)
encrypted_model_2_3_bias = encrypted_model_2_3_bias.serialize()
write_data('Local_Model_Parameters/encrypted_model_2_3_bias', encrypted_model_2_3_bias)


In [63]:
# output = dec.tolist()
# output = np.reshape(output, [200, 200])
# output = torch.as_tensor(output)
# model_dict["model0"].fc2.weight.data = output
# print(model_dict["model0"].fc2.weight[0,0:5])

In [64]:
send_data_from_local_models_to_server(model_dict, number_of_samples)